In [2]:
import os 

os.chdir('../')

In [3]:
%pwd

'a:\\Artificial intelligence\\6.Deep learning\\Chest Canser\\chest-cancer-DeepLearning-project-'

In [ ]:

# import mlflow



# with mlflow.start_run():
#     mlflow.log_params({
#         'param1': 'one',
#         'param2': 'two',
#         'param3': 'three',
#         'param4': 'four'
#     })


#     mlflow.log_metric("accuracy_score", 0.12)
#     mlflow.log_metric("precision_score", 0.23)
#     mlflow.log_metric("recall_score", 0.43)
#     mlflow.log_metric("f1", 0.89)

In [6]:
#  first 

import tensorflow as tf

model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
model

In [8]:
# 4. Update the entity 

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
# 5. Update the configuration manager in src config 

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/revanth-kumar-01-ai/chest-cancer-DeepLearning-project-.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


In [10]:
# 6. Update the components  

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import dagshub


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    dagshub.init(repo_owner='revanth-kumar-01-ai', repo_name='chest-cancer-DeepLearning-project-', mlflow=True)
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


[2025-04-16 11:15:22,002: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as revanth-kumar-01-ai

[2025-04-16 11:15:22,010: INFO: helpers: Accessing as revanth-kumar-01-ai]
[2025-04-16 11:15:22,395: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/revanth-kumar-01-ai/chest-cancer-DeepLearning-project- "HTTP/1.1 200 OK"]
[2025-04-16 11:15:22,755: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "revanth-kumar-01-ai/chest-cancer-DeepLearning-project-"

[2025-04-16 11:15:22,763: INFO: helpers: Initialized MLflow to track repo "revanth-kumar-01-ai/chest-cancer-DeepLearning-project-"]


Repository revanth-kumar-01-ai/chest-cancer-DeepLearning-project- initialized!

[2025-04-16 11:15:22,769: INFO: helpers: Repository revanth-kumar-01-ai/chest-cancer-DeepLearning-project- initialized!]


In [11]:
# 7. Update the pipeline 

try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
   print(e)

[2025-04-16 11:15:25,295: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-16 11:15:25,309: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-16 11:15:25,311: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 12s 508ms/step - loss: 9.7456 - accuracy: 0.4314
[2025-04-16 11:15:38,173: INFO: common: json file saved at: scores.json]


2025/04/16 11:15:39 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-04-16 11:15:40,594: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 13). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmp7j2lec4o\model\data\model\assets
[2025-04-16 11:15:41,019: INFO: builder_impl: Assets written to: C:\Users\User\AppData\Local\Temp\tmp7j2lec4o\model\data\model\assets]


c:\Users\User\anaconda3\envs\AIenv\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2025/04/16 11:16:43 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
